In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import openpyxl
from rdkit.Chem.Draw import IPythonConsole, MolsToGridImage

#Show mols in dataframes
from rdkit.Chem import PandasTools
from rdkit import Chem
from rdkit.Chem.Draw import MolsToGridImage
from IPython.core.display import HTML
from rdkit.Chem import Descriptors
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import rdDepictor
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
import os
import sys
from scipy.spatial.distance import pdist, squareform


In [3]:
TOP = os.getcwd().replace('notebooks', '')
raw_dir = TOP + 'data/raw/'
processed_dir = TOP +'data/processed/'
interim_dir = TOP + 'data/interim/'
external_dir = TOP + 'data/external/'
figures_dir = TOP + 'reports/figures/'

#### Read in the last file where we did an extra round of picks based on Evotec availability of the 'final picks' first identified in the Dec 2022 spreadsheet

In [4]:
df = pd.read_csv(external_dir+'gp_round2.csv', index_col = [0])

In [5]:
def tweak_df(df):
    return (df.rename(columns = {'round2_pk_gp': 'round2_picks'})
            )
            

In [6]:
df = tweak_df(df)

In [7]:
def tweak_df2(df):
    cols = ['available_round1']
    return (df.assign(available_round1=df.available_round1.replace({0: np.nan})))

In [8]:
df2 = tweak_df2(df)

In [9]:
df.loc[:,'all_picks'] = df.loc[:,['available_round1','round2_picks']].sum(axis=1, min_count=1)

In [38]:
df['all_picks'] = df['all_picks'].replace({0:np.nan})

In [39]:
df['round2_picks'].value_counts() 

1.0    174
Name: round2_picks, dtype: int64

In [40]:
df['available_round1'].value_counts() 

1.0    204
0.0    116
Name: available_round1, dtype: int64

#### Based on this - we identified a total of 378 substances (excluding the reference substances)


In [41]:
df['all_picks'].value_counts()

1.0    378
Name: all_picks, dtype: int64

In [42]:
#chk 174+204

#### What could we procure from Evotec

In [43]:
evotec = pd.read_excel(external_dir+'evotec_phase1.xlsx')

In [44]:
evotec['CUST_ID3'].value_counts()

EXACT    203
NEAR      15
Name: CUST_ID3, dtype: int64

In [45]:
evotec_ids = evotec['CUST_ID1'].tolist()

In [46]:
len(evotec_ids)

218

In [47]:
df['evotec'] = df['dtxsid'].apply(lambda x: 1 if x in evotec_ids else np.nan)

#### But only 197 substances are identified by Evotec of the 218 listed. Presumably some of these are reference substances or not overlapping due to an exact match based on DTXSID not being forthcoming?

In [48]:
df['evotec'].value_counts()

1.0    197
Name: evotec, dtype: int64

In [52]:
df[df['evotec'] == 1].groupby('group_str').size()

group_str
('Acetylides', nan)                          2
('Anthracenes', 6.0)                         1
('Anthracenes', 8.0)                         3
('Anthracenes', 9.0)                         3
('Azobenzenes', 3.0)                         1
                                            ..
('Pyridines and derivatives', 4.0)           2
('Quinolines and derivatives', 3.0)          1
('Quinolines and derivatives', 4.0)          1
('Steroids and steroid derivatives', nan)    2
('Triazines', nan)                           2
Length: 98, dtype: int64

In [53]:
df[df['all_picks']==1].groupby('group_str').size()

group_str
('Acetylides', nan)                          4
('Alkaloids and derivatives', nan)           3
('Anthracenes', 1.0)                         3
('Anthracenes', 2.0)                         3
('Anthracenes', 4.0)                         3
                                            ..
('Quinolines and derivatives', 4.0)          1
('Quinolines and derivatives', 5.0)          3
('Steroids and steroid derivatives', nan)    2
('Triazines', nan)                           3
('Triphenyl compounds', nan)                 2
Length: 141, dtype: int64

In [54]:
df2 = df.melt(id_vars = ['dtxsid', 'group_str'] ,value_vars = ['all_picks', 'evotec'])

In [61]:
df2 = df2.dropna()

In [70]:
df3 = df2.groupby(['group_str', 'variable']).size().reset_index()

In [74]:
df4 = df3.pivot_table(index = 'group_str', columns = 'variable', values = 0)

#### Just checking to see how many different categories are represented

In [76]:
df4.dropna().describe()

variable,all_picks,evotec
count,98.000000,98.000000
mean,2.571429,2.010204
std,1.184087,0.913754
min,1.000000,1.000000
25%,2.000000,1.000000
50%,2.000000,2.000000
75%,3.000000,3.000000
max,6.000000,5.000000


In [126]:
#df.columns.tolist()

#### What about if we identify the smallest categories and deprioritise those from the first plate of 75 we need to test. The 'small category' threshold here is determined by the median of the category sizes bins that have already been defined in the main spreadsheet

In [184]:
small_cats = df.groupby('group_str').size()[df.groupby('group_str').size() < 40].index.tolist()

In [185]:
len(small_cats)

95

In [186]:
df5 = df4.dropna()

In [187]:
df5 = df5.reset_index()

In [192]:
evotec_cats = df5[~df5['group_str'].isin(small_cats)]['group_str'].tolist()

#### Let's deprioritise those categories from the set of chemicals which are on the Evotec list. We will end up with more than one substance per category but we could arbitarily take the first chemical from each category. That's nets us 70 chemicals that we could plate to start with.

In [196]:
df6 = df[(df['evotec'] == 1) & (df['group_str'].isin(evotec_cats))].sort_values(by = 'group_str', ascending = True)

In [200]:
df6[['dtxsid', 'group_str', 'evotec']].drop_duplicates(subset = 'group_str', keep = 'first').to_csv(external_dir+'to_plate_first.csv')

In [201]:
df6[['dtxsid', 'group_str', 'evotec']].drop_duplicates(subset = 'group_str', keep = 'first').head()

,dtxsid,group_str,evotec
191,DTXSID4058865,"('Anthracenes', 9.0)",1.0
409,DTXSID5061457,"('Azoles', 2.0)",1.0
735,DTXSID6022054,"('Benzene and substituted derivatives', 1.0)",1.0
887,DTXSID2042436,"('Benzene and substituted derivatives', 2.0)",1.0
1432,DTXSID2021236,"('Benzene and substituted derivatives', 3.0)",1.0
